# Text generation using a RNN

Given a sequence of words from this data, train a model to predict the next word in the sequence. Longer sequences of text can be generated by calling the model repeatedly.

**Mount your Google Drive**

In [1]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


### Import Keras and other libraries

In [2]:
import glob
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional

Using TensorFlow backend.


## Download data
Reference: Data is collected from http://www.gutenberg.org

For the lab purpose, you can load the dataset provided by Great Learning

### Load the Oscar Wilde dataset

Store all the ".txt" file names in a list

In [0]:
#using glob function picked only the oscar_wilde.txt data and storing it in a variable - improper path, hence no files are retrived
df = glob.glob("/content/drive/My Drive/AIML/data/oscar_wilde/*.txt")

In [0]:
df

[]

In [0]:
#using glob function picked only the oscar_wilde data - improper path, hence no files are retrived
df2 = glob.glob("/content/drive/My Drive/AIML/data/Oscar Wilde Miscellaneous/*.txt")

In [0]:
df2

IndexError: ignored

In [0]:
#using glob function picked all the list of file and stored in a variable
df3 = glob.glob("/content/drive/My Drive/AIML/data/*.txt")

In [4]:
df3 #storing all the list of files in a variable - the file list

['/content/drive/My Drive/AIML/data/A House of Pomegranates.txt',
 '/content/drive/My Drive/AIML/data/A Woman of No Importance a play.txt',
 '/content/drive/My Drive/AIML/data/A Critic in Pall Mall.txt',
 '/content/drive/My Drive/AIML/data/An Ideal Husband.txt',
 '/content/drive/My Drive/AIML/data/Children in Prison and Other Cruelties of Prison Life.txt',
 '/content/drive/My Drive/AIML/data/Charmides and Other Poems.txt',
 '/content/drive/My Drive/AIML/data/Essays and Lectures.txt',
 '/content/drive/My Drive/AIML/data/De Profundis.txt',
 '/content/drive/My Drive/AIML/data/For Love of the King.txt',
 '/content/drive/My Drive/AIML/data/Intentions.txt',
 '/content/drive/My Drive/AIML/data/Impressions of America.txt',
 '/content/drive/My Drive/AIML/data/Lady Windermere_s Fan.txt',
 '/content/drive/My Drive/AIML/data/Selected poems of oscar wilde including The Ballad of Reading Gaol.txt',
 '/content/drive/My Drive/AIML/data/Poems with the Ballad of Reading Gaol.txt',
 '/content/drive/My Dr

### Read the data

Read contents of every file from the list and append the text in a new list

In [0]:
codetext = []
bookranges = []
for file in df:
    text = open(file, "r")
    start = len(codetext)
    codetext.append(text.read())
    end = len(codetext)
    bookranges.append({"start": start, "end": end, "name": file.rsplit("/", 1)[-1]})
    text.close()

In [0]:
len(codetext)

0

In [0]:
codetext1 = []
bookranges2 = []
for file in df2:
    text = open(file, "r")
    start = len(codetext1)
    codetext1.append(text.read())
    end = len(codetext1)
    bookranges2.append({"start": start, "end": end, "name": file.rsplit("/", 1)[-1]})
    text.close()

In [0]:
len(codetext1)

0

In [0]:
codetext1 = []
bookranges2 = []
for file in df3:
    text = open(file, "r") #opening the file from the file list path
    start = len(codetext1) #reading from the entire length of the each and every file for specifying the bookranges2 function and feeded into the same function
    codetext1.append(text.read()) #appending the entire in the codetext1 function separately
    end = len(codetext1) #ending the same - for specifying the bookranges2 function and feeded into the same function, totally we have 31 files
    bookranges2.append({"start": start, "end": end, "name": file.rsplit("/", 1)[-1]}) #this gives the start "0" <file name> end "1"
    text.close() #closing the text. close function

In [6]:
len(codetext1)

31

In [0]:
codetext1[0] #first ebook text is displayed here

'\ufeffThe Project Gutenberg eBook, A House of Pomegranates, by Oscar Wilde\n\n\nThis eBook is for the use of anyone anywhere in the United States and most\nother parts of the world at no cost and with almost no restrictions \nwhatsoever.  You may copy it, give it away or re-use it under the terms of\nthe Project Gutenberg License included with this eBook or online at \nwww.gutenberg.org.  If you are not located in the United States, you\'ll have\nto check the laws of the country where you are located before using this ebook.\n\n\n\n\nTitle: A House of Pomegranates\n\n\nAuthor: Oscar Wilde\n\n\n\nRelease Date: October 26, 2014  [eBook #873]\n[This file was first posted on April 8, 1997]\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\n\n***START OF THE PROJECT GUTENBERG EBOOK A HOUSE OF POMEGRANATES***\n\n\nTranscribed from the 1915 Methuen and Co. edition by David Price, email\nccx074@pglaf.org\n\n                          [Picture: Book cover]\n\n                             

In [0]:
codetext1[1] #second ebook text is displayed here

'\ufeffThe Project Gutenberg eBook, A Woman of No Importance, by Oscar Wilde\n\n\nThis eBook is for the use of anyone anywhere in the United States and most\nother parts of the world at no cost and with almost no restrictions \nwhatsoever.  You may copy it, give it away or re-use it under the terms of\nthe Project Gutenberg License included with this eBook or online at \nwww.gutenberg.org.  If you are not located in the United States, you\'ll have\nto check the laws of the country where you are located before using this ebook.\n\n\n\n\nTitle: A Woman of No Importance\n       A Play\n\n\nAuthor: Oscar Wilde\n\n\n\nRelease Date: September 16, 2014  [eBook #854]\n[This file was first posted on 20 March 1997]\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\n\n***START OF THE PROJECT GUTENBERG EBOOK A WOMAN OF NO IMPORTANCE***\n\n\nTranscribed from the 1919 Methuen & Co. Ltd. edition by David Price,\nemail ccx074@pglaf.org\n\n\n\n\n\n                                A WOMAN OF\n     

In [7]:
codetext1[30] #last ebook is displayed here

'\ufeffThe Project Gutenberg eBook, The Happy Prince, by Oscar Wilde, Illustrated\nby Walter Crane\n\n\nThis eBook is for the use of anyone anywhere in the United States and most\nother parts of the world at no cost and with almost no restrictions \nwhatsoever.  You may copy it, give it away or re-use it under the terms of\nthe Project Gutenberg License included with this eBook or online at \nwww.gutenberg.org.  If you are not located in the United States, you\'ll have\nto check the laws of the country where you are located before using this ebook.\n\n\n\n\nTitle: The Happy Prince\n       and Other Tales\n\n\nAuthor: Oscar Wilde\n\n\n\nRelease Date: March 29, 2015  [eBook #902]\n[This file was first posted on May 6, 1997]\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\n\n***START OF THE PROJECT GUTENBERG EBOOK THE HAPPY PRINCE***\n\n\nTranscribed from the 1910 David Nutt edition by David Price, email\nccx074@pglaf.org.  Second proof by Paul Redmond.\n\n                        

In [8]:
#bookranges2[0]
bookranges2 #numbering the books starting from 0 and ending at 1

[{'end': 1, 'name': 'A House of Pomegranates.txt', 'start': 0},
 {'end': 2, 'name': 'A Woman of No Importance a play.txt', 'start': 1},
 {'end': 3, 'name': 'A Critic in Pall Mall.txt', 'start': 2},
 {'end': 4, 'name': 'An Ideal Husband.txt', 'start': 3},
 {'end': 5,
  'name': 'Children in Prison and Other Cruelties of Prison Life.txt',
  'start': 4},
 {'end': 6, 'name': 'Charmides and Other Poems.txt', 'start': 5},
 {'end': 7, 'name': 'Essays and Lectures.txt', 'start': 6},
 {'end': 8, 'name': 'De Profundis.txt', 'start': 7},
 {'end': 9, 'name': 'For Love of the King.txt', 'start': 8},
 {'end': 10, 'name': 'Intentions.txt', 'start': 9},
 {'end': 11, 'name': 'Impressions of America.txt', 'start': 10},
 {'end': 12, 'name': 'Lady Windermere_s Fan.txt', 'start': 11},
 {'end': 13,
  'name': 'Selected poems of oscar wilde including The Ballad of Reading Gaol.txt',
  'start': 12},
 {'end': 14, 'name': 'Poems with the Ballad of Reading Gaol.txt', 'start': 13},
 {'end': 15,
  'name': 'Selected 

In [0]:
codetext1[1] 

'\ufeffThe Project Gutenberg eBook, A Woman of No Importance, by Oscar Wilde\n\n\nThis eBook is for the use of anyone anywhere in the United States and most\nother parts of the world at no cost and with almost no restrictions \nwhatsoever.  You may copy it, give it away or re-use it under the terms of\nthe Project Gutenberg License included with this eBook or online at \nwww.gutenberg.org.  If you are not located in the United States, you\'ll have\nto check the laws of the country where you are located before using this ebook.\n\n\n\n\nTitle: A Woman of No Importance\n       A Play\n\n\nAuthor: Oscar Wilde\n\n\n\nRelease Date: September 16, 2014  [eBook #854]\n[This file was first posted on 20 March 1997]\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\n\n***START OF THE PROJECT GUTENBERG EBOOK A WOMAN OF NO IMPORTANCE***\n\n\nTranscribed from the 1919 Methuen & Co. Ltd. edition by David Price,\nemail ccx074@pglaf.org\n\n\n\n\n\n                                A WOMAN OF\n     

## Process the text
Initialize and fit the tokenizer

In [0]:
tokenizer = Tokenizer(lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n') #putting numbers for each and every word in the codetext, 
                                                     #here with we are removing the special characters from getting numbered 
tokenizer.fit_on_texts(codetext1)

In [0]:
codetext1[1] 

'\ufeffThe Project Gutenberg eBook, A Woman of No Importance, by Oscar Wilde\n\n\nThis eBook is for the use of anyone anywhere in the United States and most\nother parts of the world at no cost and with almost no restrictions \nwhatsoever.  You may copy it, give it away or re-use it under the terms of\nthe Project Gutenberg License included with this eBook or online at \nwww.gutenberg.org.  If you are not located in the United States, you\'ll have\nto check the laws of the country where you are located before using this ebook.\n\n\n\n\nTitle: A Woman of No Importance\n       A Play\n\n\nAuthor: Oscar Wilde\n\n\n\nRelease Date: September 16, 2014  [eBook #854]\n[This file was first posted on 20 March 1997]\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\n\n***START OF THE PROJECT GUTENBERG EBOOK A WOMAN OF NO IMPORTANCE***\n\n\nTranscribed from the 1919 Methuen & Co. Ltd. edition by David Price,\nemail ccx074@pglaf.org\n\n\n\n\n\n                                A WOMAN OF\n     

In [0]:
tokenizer2 = Tokenizer(lower=False, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer2.fit_on_texts(codetext1)

In [0]:
codetext1[1] 

'\ufeffThe Project Gutenberg eBook, A Woman of No Importance, by Oscar Wilde\n\n\nThis eBook is for the use of anyone anywhere in the United States and most\nother parts of the world at no cost and with almost no restrictions \nwhatsoever.  You may copy it, give it away or re-use it under the terms of\nthe Project Gutenberg License included with this eBook or online at \nwww.gutenberg.org.  If you are not located in the United States, you\'ll have\nto check the laws of the country where you are located before using this ebook.\n\n\n\n\nTitle: A Woman of No Importance\n       A Play\n\n\nAuthor: Oscar Wilde\n\n\n\nRelease Date: September 16, 2014  [eBook #854]\n[This file was first posted on 20 March 1997]\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\n\n***START OF THE PROJECT GUTENBERG EBOOK A WOMAN OF NO IMPORTANCE***\n\n\nTranscribed from the 1919 Methuen & Co. Ltd. edition by David Price,\nemail ccx074@pglaf.org\n\n\n\n\n\n                                A WOMAN OF\n     

In [0]:
tokenizer3 = Tokenizer(lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')

In [0]:
tokenizer3.fit_on_texts(codetext1)

In [0]:
codetext1[1] 

'\ufeffThe Project Gutenberg eBook, A Woman of No Importance, by Oscar Wilde\n\n\nThis eBook is for the use of anyone anywhere in the United States and most\nother parts of the world at no cost and with almost no restrictions \nwhatsoever.  You may copy it, give it away or re-use it under the terms of\nthe Project Gutenberg License included with this eBook or online at \nwww.gutenberg.org.  If you are not located in the United States, you\'ll have\nto check the laws of the country where you are located before using this ebook.\n\n\n\n\nTitle: A Woman of No Importance\n       A Play\n\n\nAuthor: Oscar Wilde\n\n\n\nRelease Date: September 16, 2014  [eBook #854]\n[This file was first posted on 20 March 1997]\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\n\n***START OF THE PROJECT GUTENBERG EBOOK A WOMAN OF NO IMPORTANCE***\n\n\nTranscribed from the 1919 Methuen & Co. Ltd. edition by David Price,\nemail ccx074@pglaf.org\n\n\n\n\n\n                                A WOMAN OF\n     

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping words to numbers, and another for numbers to words.

In [0]:
word_idx = tokenizer.word_index

In [11]:
word_idx #mapping the words to numbers

{'the': 1,
 'of': 2,
 'and': 3,
 'to': 4,
 'a': 5,
 'in': 6,
 'is': 7,
 'that': 8,
 'i': 9,
 'it': 10,
 'you': 11,
 'he': 12,
 'with': 13,
 'for': 14,
 'as': 15,
 'not': 16,
 'his': 17,
 'was': 18,
 'be': 19,
 'are': 20,
 'or': 21,
 'have': 22,
 'on': 23,
 'but': 24,
 'at': 25,
 'by': 26,
 'this': 27,
 'from': 28,
 'all': 29,
 'one': 30,
 'her': 31,
 'they': 32,
 'we': 33,
 'which': 34,
 'him': 35,
 'no': 36,
 'had': 37,
 'who': 38,
 'me': 39,
 'my': 40,
 'so': 41,
 'has': 42,
 'what': 43,
 'will': 44,
 'there': 45,
 'an': 46,
 'gutenberg': 47,
 'she': 48,
 'its': 49,
 'their': 50,
 'any': 51,
 'project': 52,
 'if': 53,
 'do': 54,
 'work': 55,
 'when': 56,
 'life': 57,
 'them': 58,
 'us': 59,
 'lord': 60,
 'your': 61,
 'like': 62,
 'were': 63,
 'would': 64,
 'some': 65,
 'mr': 66,
 'art': 67,
 'very': 68,
 'more': 69,
 'man': 70,
 'can': 71,
 'may': 72,
 'been': 73,
 'about': 74,
 'out': 75,
 '’': 76,
 'should': 77,
 'than': 78,
 'our': 79,
 'great': 80,
 'tm': 81,
 'said': 82,
 'must'

In [0]:
idx_word = tokenizer.index_word #mapping the numbers to words, kind of putting index

In [13]:
idx_word

{1: 'the',
 2: 'of',
 3: 'and',
 4: 'to',
 5: 'a',
 6: 'in',
 7: 'is',
 8: 'that',
 9: 'i',
 10: 'it',
 11: 'you',
 12: 'he',
 13: 'with',
 14: 'for',
 15: 'as',
 16: 'not',
 17: 'his',
 18: 'was',
 19: 'be',
 20: 'are',
 21: 'or',
 22: 'have',
 23: 'on',
 24: 'but',
 25: 'at',
 26: 'by',
 27: 'this',
 28: 'from',
 29: 'all',
 30: 'one',
 31: 'her',
 32: 'they',
 33: 'we',
 34: 'which',
 35: 'him',
 36: 'no',
 37: 'had',
 38: 'who',
 39: 'me',
 40: 'my',
 41: 'so',
 42: 'has',
 43: 'what',
 44: 'will',
 45: 'there',
 46: 'an',
 47: 'gutenberg',
 48: 'she',
 49: 'its',
 50: 'their',
 51: 'any',
 52: 'project',
 53: 'if',
 54: 'do',
 55: 'work',
 56: 'when',
 57: 'life',
 58: 'them',
 59: 'us',
 60: 'lord',
 61: 'your',
 62: 'like',
 63: 'were',
 64: 'would',
 65: 'some',
 66: 'mr',
 67: 'art',
 68: 'very',
 69: 'more',
 70: 'man',
 71: 'can',
 72: 'may',
 73: 'been',
 74: 'about',
 75: 'out',
 76: '’',
 77: 'should',
 78: 'than',
 79: 'our',
 80: 'great',
 81: 'tm',
 82: 'said',
 83: 'm

Get the word count for every word and also get the total number of words.

In [0]:
word_counts = tokenizer.word_counts

In [15]:
word_counts

OrderedDict([('\ufeffthe', 30),
             ('project', 2634),
             ('gutenberg', 2903),
             ('ebook', 324),
             ('a', 22045),
             ('house', 623),
             ('of', 41415),
             ('pomegranates', 33),
             ('by', 5255),
             ('oscar', 182),
             ('wilde', 251),
             ('this', 4644),
             ('is', 17221),
             ('for', 8728),
             ('the', 64874),
             ('use', 743),
             ('anyone', 188),
             ('anywhere', 106),
             ('in', 18625),
             ('united', 400),
             ('states', 546),
             ('and', 35342),
             ('most', 1504),
             ('other', 1537),
             ('parts', 75),
             ('world', 1174),
             ('at', 5424),
             ('no', 3601),
             ('cost', 108),
             ('with', 9022),
             ('almost', 371),
             ('restrictions', 63),
             ('whatsoever', 99),
             ('you', 97

In [0]:
num_words = len(word_idx) + 1 #+1 is because the count starts from 0

In [17]:
num_words

32839

Convert text to sequence of numbers

In [0]:
sequences = tokenizer.texts_to_sequences(codetext1)

In [19]:
sequences #converting all the text into sequence of numbers

[[3244,
  52,
  47,
  365,
  5,
  187,
  2,
  2988,
  26,
  656,
  474,
  27,
  365,
  7,
  14,
  1,
  162,
  2,
  624,
  1079,
  6,
  1,
  289,
  215,
  3,
  87,
  85,
  1492,
  2,
  1,
  106,
  25,
  36,
  1058,
  3,
  13,
  317,
  36,
  1732,
  1145,
  11,
  72,
  273,
  10,
  142,
  10,
  154,
  21,
  1080,
  162,
  10,
  184,
  1,
  172,
  2,
  1,
  52,
  47,
  239,
  1027,
  13,
  27,
  365,
  21,
  906,
  25,
  413,
  47,
  355,
  53,
  11,
  20,
  16,
  788,
  6,
  1,
  289,
  215,
  3829,
  22,
  4,
  1176,
  1,
  266,
  2,
  1,
  304,
  122,
  11,
  20,
  788,
  133,
  722,
  27,
  365,
  1081,
  5,
  187,
  2,
  2988,
  607,
  656,
  474,
  2490,
  863,
  2598,
  3830,
  7078,
  365,
  11349,
  27,
  1335,
  18,
  127,
  848,
  23,
  1787,
  570,
  5981,
  628,
  213,
  471,
  200,
  3931,
  4824,
  570,
  938,
  2,
  1,
  52,
  47,
  365,
  5,
  187,
  2,
  2988,
  3831,
  28,
  1,
  8230,
  2133,
  3,
  698,
  483,
  26,
  1856,
  982,
  1127,
  3932,
  916,
  355,
  336,


### Generate Features and Labels

In [0]:
features = []
labels = []

training_length = 50
# Iterate through the sequences of tokens
for seq in sequences:
    # Create multiple training examples from each sequence
    for i in range(training_length, training_length+300): #taking the (0,350) nos from the above sequence generated
        # Extract the features and label
        extract = seq[i - training_length: i - training_length + 20] #splitting by 20 nos from the sequence

        # Set the features and label
        features.append(extract[:-1]) #considering upto 19 chars(0:(20-1))
        labels.append(extract[-1]) #considering the 20th chars(-1(last char))

In [21]:
features

[[3244,
  52,
  47,
  365,
  5,
  187,
  2,
  2988,
  26,
  656,
  474,
  27,
  365,
  7,
  14,
  1,
  162,
  2,
  624],
 [52,
  47,
  365,
  5,
  187,
  2,
  2988,
  26,
  656,
  474,
  27,
  365,
  7,
  14,
  1,
  162,
  2,
  624,
  1079],
 [47,
  365,
  5,
  187,
  2,
  2988,
  26,
  656,
  474,
  27,
  365,
  7,
  14,
  1,
  162,
  2,
  624,
  1079,
  6],
 [365,
  5,
  187,
  2,
  2988,
  26,
  656,
  474,
  27,
  365,
  7,
  14,
  1,
  162,
  2,
  624,
  1079,
  6,
  1],
 [5,
  187,
  2,
  2988,
  26,
  656,
  474,
  27,
  365,
  7,
  14,
  1,
  162,
  2,
  624,
  1079,
  6,
  1,
  289],
 [187,
  2,
  2988,
  26,
  656,
  474,
  27,
  365,
  7,
  14,
  1,
  162,
  2,
  624,
  1079,
  6,
  1,
  289,
  215],
 [2,
  2988,
  26,
  656,
  474,
  27,
  365,
  7,
  14,
  1,
  162,
  2,
  624,
  1079,
  6,
  1,
  289,
  215,
  3],
 [2988,
  26,
  656,
  474,
  27,
  365,
  7,
  14,
  1,
  162,
  2,
  624,
  1079,
  6,
  1,
  289,
  215,
  3,
  87],
 [26,
  656,
  474,
  27,
  365,
  7,
  

In [22]:
features[1]

[52,
 47,
 365,
 5,
 187,
 2,
 2988,
 26,
 656,
 474,
 27,
 365,
 7,
 14,
 1,
 162,
 2,
 624,
 1079]

In [23]:
labels[1]

6

In [24]:
labels

[1079,
 6,
 1,
 289,
 215,
 3,
 87,
 85,
 1492,
 2,
 1,
 106,
 25,
 36,
 1058,
 3,
 13,
 317,
 36,
 1732,
 1145,
 11,
 72,
 273,
 10,
 142,
 10,
 154,
 21,
 1080,
 162,
 10,
 184,
 1,
 172,
 2,
 1,
 52,
 47,
 239,
 1027,
 13,
 27,
 365,
 21,
 906,
 25,
 413,
 47,
 355,
 53,
 11,
 20,
 16,
 788,
 6,
 1,
 289,
 215,
 3829,
 22,
 4,
 1176,
 1,
 266,
 2,
 1,
 304,
 122,
 11,
 20,
 788,
 133,
 722,
 27,
 365,
 1081,
 5,
 187,
 2,
 2988,
 607,
 656,
 474,
 2490,
 863,
 2598,
 3830,
 7078,
 365,
 11349,
 27,
 1335,
 18,
 127,
 848,
 23,
 1787,
 570,
 5981,
 628,
 213,
 471,
 200,
 3931,
 4824,
 570,
 938,
 2,
 1,
 52,
 47,
 365,
 5,
 187,
 2,
 2988,
 3831,
 28,
 1,
 8230,
 2133,
 3,
 698,
 483,
 26,
 1856,
 982,
 1127,
 3932,
 916,
 355,
 336,
 199,
 2717,
 4,
 5982,
 1733,
 474,
 5,
 187,
 2,
 2988,
 26,
 656,
 474,
 2133,
 698,
 3531,
 4475,
 5214,
 594,
 1043,
 427,
 396,
 3378,
 483,
 127,
 781,
 4056,
 127,
 4187,
 26,
 2133,
 3,
 698,
 515,
 674,
 23,
 2904,
 5983,
 711,
 3,
 1876,
 371

In [25]:
len(labels)

9300

In [26]:
len(features)

9300

The count of features and labels are same, as well took first 20 chars from the sequence and splitted 19(first 19 chars) - for features of the 1st sequence,20th char of the 1 sequnce for lables - hence the count of the lables and features remains the same

### The prediction task

Given a word, or a sequence of words, what is the most probable next word? This is the task we're training the model to perform. The input to the model will be a sequence of words, and we train the model to predict the output—the following word at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the words computed until this moment, what is the next word?

### Generate training and testing data

In [0]:
from sklearn.utils import shuffle
import numpy as np

features, labels = shuffle(features, labels, random_state=1)

# Decide on number of samples for training
train_end = int(0.7 * len(labels)) #splitting the 70:30 ratio as both the count are same

train_features = np.array(features[:train_end]) #splitting the features - 70% of the data to train, using the ":"
valid_features = np.array(features[train_end:]) #splitting the remaining features - 30% of the data to valid(test feature) ":"

train_labels = labels[:train_end] #splitting the labels - 70% of the data to train, using the ":"
valid_labels = labels[train_end:] #splitting the remaining lables - 30% of the data to valid(test feature) ":"

# Convert to arrays
X_train, X_valid = np.array(train_features), np.array(valid_features)

# Using int8 for memory savings
y_train = np.zeros((len(train_labels), num_words), dtype=np.int8) #creating y_train and y_valied by getting the length of the train labels(input shape) and
                                                                  # matching it with the num_Words(length 32389) (input shape) 
                                                                  #https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.zeros.html
                                                                  #filling the values with zeros and storing it as int8 dtype format
y_valid = np.zeros((len(valid_labels), num_words), dtype=np.int8)

# One hot encoding of labels
for example_index, word_index in enumerate(train_labels):  #enumerating the y_train to y_valid to the corresponding train_lables and valid_lables
    y_train[example_index, word_index] = 1            #suppose if the 6th value in the sequnce is 14 then 6th value will be converted as 0 and remaining
                                                      #32388 values will be as zeros(0s) 

for example_index, word_index in enumerate(valid_labels):
    y_valid[example_index, word_index] = 1

In [0]:
X_train[0]

array([    2,     1,  5866,  1868,     2,   331,     4,   691, 16330,
           1,  5474,     2,     1,   864,    76,    15,  1370,  1546,
          10])

In [0]:
X_valid[0]

array([ 8396, 10703, 21147,     3,     1,   906,   880,  7407,  5385,
          25,   835,   413,  7965,   995,     1,   336,     2,   253,
         473])

In [0]:
y_train #one hot encoding of the variables

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [0]:
len(y_train[0])

32839

In [0]:
len(y_train)

6510

In [0]:
y_train.dtype

dtype('int8')

In [0]:
len(y_valid[0])

32839

In [0]:
len(y_valid)

2790

This is just to check the features and labels

In [28]:
for i, sequence in enumerate(X_train[:2]):
    text = []
#     print(i, sequence)
    for idx in sequence:
        text.append(idx_word[idx])
        
    print('Features: ' + ' '.join(text)+'\n')
    print('Label: ' + idx_word[np.argmax(y_train[i])] + '\n')

Features: of the clamorous claims of others to stand ‘under the shelter of the wall ’ as plato puts it

Label: and

Features: project gutenberg ebook essays and lectures transcribed from the 1913 methuen and co edition by david price email ccx074

Label: pglaf



In [29]:
for i, sequence in enumerate(X_train[:4]):
    text = []
#     print(i, sequence)
    for idx in sequence:
        text.append(idx_word[idx])
        
    print('Features: ' + ' '.join(text)+'\n')
    print('Label: ' + idx_word[np.argmax(y_train[i])] + '\n')

Features: of the clamorous claims of others to stand ‘under the shelter of the wall ’ as plato puts it

Label: and

Features: project gutenberg ebook essays and lectures transcribed from the 1913 methuen and co edition by david price email ccx074

Label: pglaf

Features: more apparent than in some of his finished work what i believed to be only the fragment of an

Label: essay

Features: afternoon tea 76 early christian art in ireland 81 madame ristori 85 english poetesses 91 venus or victory 101

Label: m



## Build The Model

Use `keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `keras.layers.LSTM`: A type of RNN with size `units=rnn_units` (You can also use a GRU layer here.)
* `keras.layers.Dense`: The output layer, with `num_words` outputs.

In [30]:
model2 = Sequential()

# Embedding layer
model2.add(
    Embedding(
        input_dim=len(word_idx) + 1,
        output_dim=100,
        weights=None,
        trainable=True))

# Recurrent layer
model2.add(
    LSTM(
        64, return_sequences=False, dropout=0.1,
        recurrent_dropout=0.1))

# Fully connected layer
model2.add(Dense(64, activation='relu'))

# Dropout for regularization
model2.add(Dropout(0.5))

# Output layer
model2.add(Dense(num_words, activation='softmax'))

# Compile the model
model2.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         3283900   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32839)             2134535   
Total params: 5,464,835
Trainable params: 5,464,835
Non-trainable params: 0
_______________________________________________________

For each word the model looks up the embedding, runs the LSTM one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next word.

## Train the model

In [32]:
M = model2.fit(X_train, y_train, epochs = 10, batch_size = 2000, 
          validation_data = (X_valid, y_valid), 
          verbose = 1)## Train the model

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 6510 samples, validate on 2790 samples
Epoch 1/10





6510/6510 [==============================] - 12s 2ms/step - loss: 10.3976 - acc: 0.0052 - val_loss: 10.3927 - val_acc: 0.0348
Epoch 2/10
6510/6510 [==============================] - 1s 218us/step - loss: 10.3888 - acc: 0.0301 - val_loss: 10.3827 - val_acc: 0.0466
Epoch 3/10
6510/6510 [==============================] - 1s 218us/step - loss: 10.3747 - acc: 0.0330 - val_loss: 10.3610 - val_acc: 0.0577
Epoch 4/10
6510/6510 [==============================] - 1s 218us/step - loss: 10.3407 - acc: 0.0324 - val_loss: 10.2964 - val_acc: 0.0620
Epoch 5/10
6510/6510 [==============================] - 1s 217us/step - loss: 10.2305 - acc: 0.0309 - val_loss: 10.0628 - val_acc: 0.0620
Epoch 6/10
6510/6510 [==============================] - 1s 218us/step - loss: 9.8462 - acc: 0.0367 - val_loss: 9.3833 - val_acc: 0.0620
Epoch 7/10
6510/6510 [===

### Save Model

In [0]:
# save the model to file
model2.save('seq_nlp_assignment.h5')

## If you have already trained the model and saved it, you can load a pretrained model

In [0]:
# load the model
model = load_model('seq_nlp_assignment.h5')

### Note: After loading the model run  model.fit()  to continue training form there, if required.

In [35]:
model.fit(X_train, y_train, batch_size=500, epochs=5)

Epoch 1/5
6510/6510 [==============================] - 2s 254us/step - loss: 6.4363 - acc: 0.0619
Epoch 2/5
6510/6510 [==============================] - 1s 161us/step - loss: 6.2947 - acc: 0.0630
Epoch 3/5
6510/6510 [==============================] - 1s 163us/step - loss: 6.2702 - acc: 0.0631
Epoch 4/5
6510/6510 [==============================] - 1s 162us/step - loss: 6.2628 - acc: 0.0539
Epoch 5/5
6510/6510 [==============================] - 1s 162us/step - loss: 6.2418 - acc: 0.0559


## Evaluation

In [36]:
print(model.evaluate(X_train, y_train, batch_size = 20))
print('\nModel Performance: Log Loss and Accuracy on validation data')
print(model.evaluate(X_valid, y_valid, batch_size = 20))

6510/6510 [==============================] - 4s 559us/step
[6.14643338418776, 0.06298003206101065]

Model Performance: Log Loss and Accuracy on validation data
2790/2790 [==============================] - 1s 517us/step
[7.356258669207173, 0.06200716970321525]


## Generate text

In [37]:
seed_length=50
new_words=50
diversity=1
n_gen=1

import random

# Choose a random sequence
seq = random.choice(sequences)

# print seq

# Choose a random starting point
seed_idx = random.randint(0, len(seq) - seed_length - 10)
# Ending index for seed
end_idx = seed_idx + seed_length

gen_list = []

for n in range(n_gen):
    # Extract the seed sequence
    seed = seq[seed_idx:end_idx]
    original_sequence = [idx_word[i] for i in seed]
    generated = seed[:] + ['#']

    # Find the actual entire sequence
    actual = generated[:] + seq[end_idx:end_idx + new_words]
        # Keep adding new words
    for i in range(new_words):

        # Make a prediction from the seed
        preds = model.predict(np.array(seed).reshape(1, -1))[0].astype(np.float64)

        # Diversify
        preds = np.log(preds) / diversity
        exp_preds = np.exp(preds)

        # Softmax
        preds = exp_preds / sum(exp_preds)

        # Choose the next word
        probas = np.random.multinomial(1, preds, 1)[0]

        next_idx = np.argmax(probas)

        # New seed adds on old word
        #             seed = seed[1:] + [next_idx]
        seed += [next_idx]
        generated.append(next_idx)
    # Showing generated and actual abstract
    n = []

    for i in generated:
        n.append(idx_word.get(i, '< --- >'))

    gen_list.append(n)
    a = []

for i in actual:
    a.append(idx_word.get(i, '< --- >'))

a = a[seed_length:]

gen_list = [gen[seed_length:seed_length + len(a)] for gen in gen_list]

print('Original Sequence: \n'+' '.join(original_sequence))
print("\n")
# print(gen_list)
print('Generated Sequence: \n'+' '.join(gen_list[0][1:]))
# print(a)

Original Sequence: 
barren but once we fancy the white feet of the muses brushed the dew from the anemones in the morning and at evening came apollo to sing to the shepherds in the vale but in this we are merely lending to other ages what we desire or think we desire


Generated Sequence: 
of 1908 16 some fall english appeared ebook secret archive anyone love magdalen the spite outside there project gaol day now house on character have license city internet other ross around or not ’ email was is ccx074 poet the the mrs statue it ascii around on from email at


##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.



---



---

